In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="RECALL"
cutoff_to_optimize=20

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[20], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[20], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

recommenderClass = MultiThreadSLIM_SLIMElasticNetRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
slim_study = optuna.load_study(study_name="SLIM_XG_RECALL", storage=storage)
rp3_study = optuna.load_study(study_name="RP3_XG_RECALL", storage=storage)

In [15]:
slims = []
rp3s = []

for i in range(len(seeds)):
    slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train[i])
    slim.fit(workers=8, **slim_study.best_params)
    slims.append(slim)

    rp3 = RP3betaRecommender(urm_train[i])
    rp3.fit(**rp3_study.best_params)
    rp3s.append(rp3)


SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [01:09<00:00, 321.75it/s]


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4450.92 column/sec. Elapsed time 4.99 sec
SLIMElasticNetRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 325 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [01:13<00:00, 303.85it/s]


RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4441.42 column/sec. Elapsed time 5.00 sec
SLIMElasticNetRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

RP3betaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5187.04 column/sec. Elapsed time 4.28 sec
SLIMElasticNetRecommender: URM Detected 478 ( 3.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 329 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [01:09<00:00, 320.40it/s]


RP3betaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4208.92 column/sec. Elapsed time 5.28 sec
SLIMElasticNetRecommender: URM Detected 460 ( 3.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 348 ( 1.6%) items with no interactions.


100%|█████████▉| 22216/22222 [01:10<00:00, 315.42it/s]


RP3betaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4221.86 column/sec. Elapsed time 5.26 sec


In [16]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender


hybrids = []

for i in range(len(seeds)):
    hybrid = ItemKNNSimilarityHybridRecommender(urm_train[i], slims[i].W_sparse, rp3s[i].W_sparse)
    hybrids.append(hybrid)

ItemKNNSimilarityHybridRecommender: URM Detected 508 ( 4.0%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 332 ( 1.5%) items with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 440 ( 3.5%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 325 ( 1.5%) items with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 479 ( 3.8%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 451 ( 3.6%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 311 ( 1.4%) items with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 449 ( 3.6%) users with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 320 ( 1.4%) items with no interactions.
ItemKNNSimilarityHybridRecommender: URM Detected 460 ( 3.6%) users with no interactions.
ItemKNNSimilarityHybr

In [17]:
study = optuna.create_study(study_name="HYB_XG_RECALL", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-10 11:57:22,643] A new study created in RDB with name: HYB_XG_RECALL


In [24]:
def objective(trial):

    params = {
        'alpha': trial.suggest_float('alpha', 0, 1),
        'topK': trial.suggest_int('topK', 5, 10000),
    }

    RECALL = 0
    MAP = 0
    for i in range(k_fold):
        recommender = hybrids[i]
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
        
        RECALL += result_dict[metric_to_optimize].item()
        MAP += result_dict["MAP"].item()
    print(f"RECALL: {RECALL / k_fold}, MAP: {MAP / k_fold}")
    RECALL /= k_fold
    return RECALL

In [26]:
study.optimize(objective, n_trials=200)

EvaluatorHoldout: Processed 10057 (100.0%) in 4.77 sec. Users per second: 2109
EvaluatorHoldout: Processed 10040 (100.0%) in 6.60 sec. Users per second: 1522
EvaluatorHoldout: Processed 10017 (100.0%) in 6.24 sec. Users per second: 1604
EvaluatorHoldout: Processed 10037 (100.0%) in 6.91 sec. Users per second: 1454
EvaluatorHoldout: Processed 9989 (100.0%) in 5.15 sec. Users per second: 1941
EvaluatorHoldout: Processed 10024 (100.0%) in 5.20 sec. Users per second: 1929
EvaluatorHoldout: Processed 10030 (100.0%) in 6.65 sec. Users per second: 1509
EvaluatorHoldout: Processed 10052 (100.0%) in 5.77 sec. Users per second: 1743
EvaluatorHoldout: Processed 10045 (100.0%) in 5.95 sec. Users per second: 1690
EvaluatorHoldout: Processed 10068 (100.0%) in 5.83 sec. Users per second: 1727
RECALL: 0.16871277065181814, MAP: 0.017862709071001598


[I 2024-01-10 12:29:12,154] Trial 8 finished with value: 0.16871277065181814 and parameters: {'alpha': 0.28492450229616595, 'topK': 6469}. Best is trial 8 with value: 0.16871277065181814.


EvaluatorHoldout: Processed 10057 (100.0%) in 6.52 sec. Users per second: 1542
EvaluatorHoldout: Processed 10040 (100.0%) in 6.13 sec. Users per second: 1639
EvaluatorHoldout: Processed 10017 (100.0%) in 5.89 sec. Users per second: 1700
EvaluatorHoldout: Processed 10037 (100.0%) in 4.85 sec. Users per second: 2069
EvaluatorHoldout: Processed 9989 (100.0%) in 6.20 sec. Users per second: 1610
EvaluatorHoldout: Processed 10024 (100.0%) in 6.00 sec. Users per second: 1672
EvaluatorHoldout: Processed 10030 (100.0%) in 6.62 sec. Users per second: 1514
EvaluatorHoldout: Processed 10052 (100.0%) in 5.44 sec. Users per second: 1848
EvaluatorHoldout: Processed 10045 (100.0%) in 6.44 sec. Users per second: 1560
EvaluatorHoldout: Processed 10068 (100.0%) in 5.72 sec. Users per second: 1761
RECALL: 0.16795341529963667, MAP: 0.017901728730607486


[I 2024-01-10 12:31:46,499] Trial 9 finished with value: 0.16795341529963667 and parameters: {'alpha': 0.4267360759382067, 'topK': 2428}. Best is trial 8 with value: 0.16871277065181814.


EvaluatorHoldout: Processed 10057 (100.0%) in 5.63 sec. Users per second: 1787
EvaluatorHoldout: Processed 10040 (100.0%) in 5.72 sec. Users per second: 1755
EvaluatorHoldout: Processed 10017 (100.0%) in 5.59 sec. Users per second: 1792
EvaluatorHoldout: Processed 10037 (100.0%) in 5.89 sec. Users per second: 1705
EvaluatorHoldout: Processed 9989 (100.0%) in 5.39 sec. Users per second: 1853
EvaluatorHoldout: Processed 10024 (100.0%) in 5.89 sec. Users per second: 1701
EvaluatorHoldout: Processed 10030 (100.0%) in 6.22 sec. Users per second: 1614
EvaluatorHoldout: Processed 10052 (100.0%) in 6.82 sec. Users per second: 1474
EvaluatorHoldout: Processed 10045 (100.0%) in 6.36 sec. Users per second: 1579
EvaluatorHoldout: Processed 10068 (100.0%) in 6.08 sec. Users per second: 1657
RECALL: 0.16618666908439633, MAP: 0.017230617158283706


...

EvaluatorHoldout: Processed 10057 (100.0%) in 6.59 sec. Users per second: 1527
EvaluatorHoldout: Processed 10040 (100.0%) in 5.20 sec. Users per second: 1931
EvaluatorHoldout: Processed 10017 (100.0%) in 6.02 sec. Users per second: 1665
EvaluatorHoldout: Processed 10037 (100.0%) in 6.62 sec. Users per second: 1515
EvaluatorHoldout: Processed 9989 (100.0%) in 6.42 sec. Users per second: 1557
EvaluatorHoldout: Processed 10024 (100.0%) in 6.17 sec. Users per second: 1624
EvaluatorHoldout: Processed 10030 (100.0%) in 5.82 sec. Users per second: 1724
EvaluatorHoldout: Processed 10052 (100.0%) in 6.42 sec. Users per second: 1567
EvaluatorHoldout: Processed 10045 (100.0%) in 5.65 sec. Users per second: 1777
EvaluatorHoldout: Processed 10068 (100.0%) in 6.83 sec. Users per second: 1475
RECALL: 0.16854157672050765, MAP: 0.017866550993223585


[I 2024-01-10 14:07:55,488] Trial 47 finished with value: 0.16854157672050765 and parameters: {'alpha': 0.31017738096582115, 'topK': 8803}. Best is trial 21 with value: 0.16892795724841386.


EvaluatorHoldout: Processed 10057 (100.0%) in 6.51 sec. Users per second: 1545
EvaluatorHoldout: Processed 10040 (100.0%) in 6.43 sec. Users per second: 1561
EvaluatorHoldout: Processed 10017 (100.0%) in 6.54 sec. Users per second: 1531


[W 2024-01-10 14:08:49,689] Trial 48 failed with parameters: {'alpha': 0.764790961177337, 'topK': 9982} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_409090/760514268.py", line 12, in objective
    recommender.fit(**params)
  File "/home/japo/RecSys-Challenge/Recommenders/KNN/ItemKNNSimilarityHybridRecommender.py", line 43, in fit
    self.W_sparse = similarityMatrixTopK(W_sparse, k=self.topK)
  File "/home/japo/RecSys-Challenge/Recommenders/Recommender_utils.py", line 121, in similarityMatrixTopK
    similarity_builder.add_data_lists(row_list_to_add = column_row_index,
  File "/home/japo/RecSys-Challenge/Recommenders/Similarity/Compute_Similarity_Python.py", line 54, in add_data_lists
    self._data_array[self._next_cell_pointer] = data_list_to_add[data_point

KeyboardInterrupt: 

In [33]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(workers=8, **study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict[metric_to_optimize].item()

best_MAP_test /= k_fold
best_MAP_test

SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


KeyboardInterrupt: 

In [ ]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(workers=8,**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict[metric_to_optimize].item()

best_MAP_test /= k_fold
best_MAP_test

SLIMElasticNetRecommender: URM Detected 249 ( 2.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 113 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:47<00:00, 472.17it/s]


EvaluatorHoldout: Processed 10492 (100.0%) in 5.18 sec. Users per second: 2027
SLIMElasticNetRecommender: URM Detected 191 ( 1.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 101 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:43<00:00, 506.13it/s]


EvaluatorHoldout: Processed 10461 (100.0%) in 4.85 sec. Users per second: 2157
SLIMElasticNetRecommender: URM Detected 235 ( 1.9%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 105 ( 0.5%) items with no interactions.


...

100%|█████████▉| 22216/22222 [00:34<00:00, 648.77it/s]


EvaluatorHoldout: Processed 10498 (100.0%) in 3.95 sec. Users per second: 2658
SLIMElasticNetRecommender: URM Detected 217 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 113 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:34<00:00, 640.60it/s]


EvaluatorHoldout: Processed 10451 (100.0%) in 3.83 sec. Users per second: 2726


0.04939886597488928